In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from category_encoders import TargetEncoder
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from scipy.stats import chisquare

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
pd.set_option('display.max_colwidth', 100)
pd.set_option('expand_frame_repr', True)



/kaggle/input/microsoft-security-incident-prediction/GUIDE_Train.csv
/kaggle/input/microsoft-security-incident-prediction/GUIDE_Test.csv


def chisquare_test(f_obs, f_exp, levels_of_group_by, target_unique_values):
    for index in levels_of_group_by:
        number_of_observations = f_obs[index].sum()
        f_exp_chi_all = round(f_exp * number_of_observations)
        f_exp_chi = []
        f_obs_chi = []
        for target in target_unique_values:
            f_exp_chi.append(f_exp_chi_all[target])
            f_obs_chi.append(f_obs[index][target])
        # print(f_exp_chi, f_obs_chi)
        print(f"For {index}")
        # print(f_exp_chi_all, f_obs[index])
        print(chisquare(f_exp = f_exp_chi, f_obs = f_obs[index]))

In [3]:
def remove_right_side_up_to_dot_no_duplicates(strings):
    seen = set()
    result = []
    for s in strings:
        last_dot_index = s.rfind('.')
        if last_dot_index != -1:
            processed = s[:last_dot_index]
        else:
            processed = s
        if processed not in seen:
            seen.add(processed)
            result.append(processed)
    return result

In [4]:
def device_transformer(df):
    df['devicebrand'] = df['devicebrand'].replace(df['devicebrand'].unique()[df['devicebrand'].value_counts() < df['devicebrand'].value_counts()[3]],'Other')
    print(df['devicebrand'].value_counts())
    return df

In [6]:
df_train = pd.read_csv("/kaggle/input/microsoft-security-incident-prediction/GUIDE_Train.csv")
df_test = pd.read_csv("/kaggle/input/microsoft-security-incident-prediction/GUIDE_Test.csv")

/tmp/ipykernel_96/298725254.py:2: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df_test = pd.read_csv("/kaggle/input/microsoft-security-incident-prediction/GUIDE_Test.csv")


In [ ]:
df_train.info()
df_test.info()

In [7]:
change_to_object = ["OrgId","IncidentId", "AlertId"]
for col_name in change_to_object:
    df_train[col_name] = df_train[col_name].astype("object")
    df_test[col_name] = df_test[col_name].astype("object")     

In [8]:
df_train["Timestamp"] = pd.to_datetime(df_train["Timestamp"])
df_test["Timestamp"] = pd.to_datetime(df_test["Timestamp"])
df_train["Timestamp"]

0         2024-06-04 06:05:15+00:00
1         2024-06-14 03:01:25+00:00
2         2024-06-13 04:52:55+00:00
3         2024-06-10 16:39:36+00:00
4         2024-06-15 01:08:07+00:00
                     ...           
9516832   2024-06-08 15:26:08+00:00
9516833   2024-06-06 09:40:08+00:00
9516834   2024-06-11 22:51:03+00:00
9516835   2024-05-20 23:01:01+00:00
9516836   2024-06-03 23:51:52+00:00
Name: Timestamp, Length: 9516837, dtype: datetime64[ns, UTC]

In [9]:
components = ['year', 'month', 'day', 'hour', 'minute', 'second']
for component in components:
    df_train[component] = getattr(df_train['Timestamp'].dt, component)
    df_test[component] = getattr(df_test['Timestamp'].dt, component)

after splitting timestamp into more columns we can now drop the columns original column


In [10]:
df_train = df_train.drop(labels = ["Timestamp"], axis = 1)
df_test = df_test.drop(labels = ["Timestamp"], axis = 1)

In [10]:
print(df_test.shape, df_train.shape)
df_test.columns.difference(df_train.columns)

(4147992, 51) (9516837, 50)


Index(['Usage'], dtype='object')

Due to the difference in validation and trainig. Set we have to drop a column which is not included in traing set

In [11]:
df_test = df_test.drop(columns = ["Usage"],axis = 1)
# X_test = X_test.drop(columns = ["Usage"])

In [12]:
print(df_test.shape[0], df_train.shape[0])

4147992 9516837


Find out the number of missing values in columns


In [13]:
for i in df_train.columns:
    print(f"train {i} : {df_train[i].isna().sum()/train_shape}")
    print(f"test {i} : {df_test[i].isna().sum()/test_shape}")
    

NameError: name 'train_shape' is not defined

For now we will drop the columns with high percent of null values. *"IncidentGrade"* has a few missing values in training set however None in test set it might be caused by random chance. In terms of rest of columns it looks like a Null value has a meaning in each column and it should be later resolved. Columns *"ActionGrouped"* and *"ActionGranular"* has the exact same percantages, it might be optimall to merge this columns once I will want to handle the meaningfull NuLL values  

In [12]:
df_train = df_train.dropna(subset = ["IncidentGrade"],axis = 0)


In [13]:
low_cardinality_cols = ["NetworkMessageId", "DeviceName","RegistryKey","RegistryValueName", "RegistryValueData", "ApplicationId","ApplicationName", "OAuthApplicationId", "FolderPath", "FileName", "ResourceIdName", "OSFamily", "OSVersion", "CountryCode", "State", "City"] 

In [15]:
for col in low_cardinality_cols:
    print(f"{col}:\n {df_train[col].value_counts()}")

NetworkMessageId:
 NetworkMessageId
529644    8306658
0            2733
3             850
1             737
7             700
           ...   
246712          1
311483          1
205739          1
323861          1
484365          1
Name: count, Length: 375196, dtype: int64
DeviceName:
 DeviceName
153085    8769875
0            4376
1            3944
5            2153
4            2152
           ...   
51511           1
66905           1
136860          1
94537           1
98847           1
Name: count, Length: 114267, dtype: int64
RegistryKey:
 RegistryKey
1631    9447973
0         11076
1           245
3           235
6           141
         ...   
580           1
1406          1
1141          1
1197          1
1585          1
Name: count, Length: 1341, dtype: int64
RegistryValueName:
 RegistryValueName
635    9461017
0          483
1          483
5          311
6          256
        ...   
446          1
527          1
369          1
572          1
471          1
Name: count, Le

OAuthApplicationId, ResourceIdName, RegistryValueName and RegistryValueData are all low cardinality (almost all records are single values only) columns so I would OPT for deleing them from our Model. 

In [14]:
df_train = df_train.drop(labels = ["OAuthApplicationId", "ResourceIdName", "RegistryValueName", "RegistryValueData"], axis = 1)
df_test = df_test.drop(columns = ["OAuthApplicationId", "ResourceIdName", "RegistryValueName", "RegistryValueData"])


In [15]:
df_train.groupby("OSFamily")["OSVersion"].value_counts()

OSFamily  OSVersion
0         0             186093
          1               1636
          4                732
          11                66
          16                38
          13                32
          14                31
          15                22
          17                17
          28                 5
          31                 4
          35                 4
          29                 3
          37                 3
          38                 3
          23                 2
          30                 2
          34                 2
          43                 2
          44                 2
          47                 1
          48                 1
          51                 1
          52                 1
          53                 1
          56                 1
          64                 1
          65                 1
1         2               1884
          6                359
          8                132
          9        

As we can see most of the data is coming from two OSFamilies and their most used OSVersion. Therefore from this column we can leave just these two values and rest group to the value called 'other', before that however we have to handle the duplicate fo OSVersion in OSFamily 3.   

In [16]:
df_train["OSVersion"][df_train["OSFamily"] == 3] = 67
df_test["OSVersion"][df_test["OSFamily"] == 3] = 67

/tmp/ipykernel_96/3696779759.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_train["OSVersion"][df_train["OSFamily"] == 3] = 67
/tmp/ipykernel_96/3696779759.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

Let's see wether there are more duplicates for values of the biggest importance

In [17]:
print(df_train["OSFamily"][df_train["OSVersion"] == 66].value_counts(), "\n")
print(df_train["OSFamily"][df_train["OSVersion"] == 0].value_counts(), "\n")

OSFamily
5    9272568
Name: count, dtype: int64 

OSFamily
0    186093
Name: count, dtype: int64 



In [18]:
df_train.dtypes

Id                     int64
OrgId                 object
IncidentId            object
AlertId               object
DetectorId             int64
AlertTitle             int64
Category              object
MitreTechniques       object
IncidentGrade         object
ActionGrouped         object
ActionGranular        object
EntityType            object
EvidenceRole          object
DeviceId               int64
Sha256                 int64
IpAddress              int64
Url                    int64
AccountSid             int64
AccountUpn             int64
AccountObjectId        int64
AccountName            int64
DeviceName             int64
NetworkMessageId       int64
EmailClusterId       float64
RegistryKey            int64
ApplicationId          int64
ApplicationName        int64
ThreatFamily          object
FileName               int64
FolderPath             int64
ResourceType          object
Roles                 object
OSFamily               int64
OSVersion              int64
AntispamDirect

Now change less common OS FAMilies and Versions to the VAlue OTHER

In [19]:
# df_train[df_train["OSFamily"] != [0,5] ] = "Other"
df_train.loc[~df_train['OSFamily'].isin([0, 5]), 'OSFamily'] = 'Other'
df_train.loc[~df_train['OSVersion'].isin([66,0]), 'OSVersion'] = 'Other'
df_test.loc[~df_test['OSFamily'].isin([0, 5]), 'OSFamily'] = 'Other'
df_test.loc[~df_test['OSVersion'].isin([66,0]), 'OSVersion'] = 'Other'

/tmp/ipykernel_96/3872579750.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Other' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_train.loc[~df_train['OSFamily'].isin([0, 5]), 'OSFamily'] = 'Other'
/tmp/ipykernel_96/3872579750.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Other' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_train.loc[~df_train['OSVersion'].isin([66,0]), 'OSVersion'] = 'Other'
/tmp/ipykernel_96/3872579750.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Other' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_test.loc[~df_test['OSFamily'].isin([0, 5]), 'OSFamily'] = 'Other'
/tmp/ip

In [ ]:
df_train["OSVersion"] = df_train["OSVersion"].astype('category')
df_train["OSFamily"] = df_train["OSFamily"].astype('category')
df_test["OSVersion"] = df_test["OSVersion"].astype('category')
df_test["OSFamily"] = df_test["OSFamily"].astype('category')

In [21]:
df_train.groupby("OSFamily")["OSVersion"].value_counts()

/tmp/ipykernel_31/3660354893.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_train.groupby("OSFamily")["OSVersion"].value_counts()


OSFamily  OSVersion
0         0             186093
          Other           2614
          66                 0
5         66           9272568
          0                  0
          Other              0
Other     Other           4222
          0                  0
          66                 0
Name: count, dtype: int64

Now deletering columns with high number of Null values

In [20]:
null_percentage_train = df_train.isnull().sum() * 100

# Set your threshold (e.g., 30%)
threshold = 30
high_null_cols = null_percentage_train[null_percentage_train > threshold].index.tolist()
high_null_cols

['MitreTechniques',
 'ActionGrouped',
 'ActionGranular',
 'EmailClusterId',
 'ThreatFamily',
 'ResourceType',
 'Roles',
 'AntispamDirection',
 'SuspicionLevel',
 'LastVerdict']

popping SuspicionLevel from columns to drop due to High NULL values Based on my previous analysis 

In [21]:
high_null_cols.remove("SuspicionLevel")
high_null_cols

['MitreTechniques',
 'ActionGrouped',
 'ActionGranular',
 'EmailClusterId',
 'ThreatFamily',
 'ResourceType',
 'Roles',
 'AntispamDirection',
 'LastVerdict']

In [28]:
df_train['MitreTechniques'].value_counts()

MitreTechniques
T1078;T1078.004    1354179
T1566.002           813679
T1566               656293
T1133               145579
T1566.001           136888
                    ...   
T0814;T0816              1
T0838;T0878              1
T1567;T1105              1
T1084                    1
T0831;T0837              1
Name: count, Length: 1162, dtype: int64

In [29]:
aux_SER = df_train['MitreTechniques']
aux_SER.dtype
aux_SER = aux_SER.fillna(value = "None")

In [30]:
aux_SER = aux_SER.str.split(pat = ";")
aux_SER.value_counts()

MitreTechniques
[None]                   5427513
[T1078, T1078.004]       1354179
[T1566.002]               813679
[T1566]                   656293
[T1133]                   145579
                          ...   
[T1587, T1583, T1588]          1
[T0831, T0858]                 1
[T1595, T1464]                 1
[T0814, T0816]                 1
[T1556, T1098]                 1
Name: count, Length: 1163, dtype: int64

In [31]:
aux_SER = aux_SER.apply(remove_right_side_up_to_dot_no_duplicates)
aux_SER.value_counts()

MitreTechniques
[None]                   5427513
[T1566]                  1606962
[T1078]                  1384161
[T1110]                   152188
[T1133]                   145579
                          ...   
[T1098, T1078, T0859]          1
[T1059, T1218, T1216]          1
[T1587, T1583, T1588]          1
[T0838, T0878]                 1
[T0831, T0858]                 1
Name: count, Length: 849, dtype: int64

In [32]:
mlb = MultiLabelBinarizer()
one_hot_encoded = mlb.fit_transform(aux_SER)

# Get the feature names (unique techniques)
feature_names = mlb.classes_


In [33]:
print(one_hot_encoded, "\n")
print((feature_names))

[[1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]] 

['None' 'T0807' 'T0814' 'T0816' 'T0817' 'T0820' 'T0823' 'T0826' 'T0827'
 'T0828' 'T0829' 'T0831' 'T0836' 'T0837' 'T0838' 'T0840' 'T0843' 'T0849'
 'T0852' 'T0853' 'T0855' 'T0857' 'T0858' 'T0859' 'T0860' 'T0863' 'T0865'
 'T0866' 'T0871' 'T0873' 'T0878' 'T0885' 'T0886' 'T0889' 'T0890' 'T1001'
 'T1003' 'T1005' 'T1006' 'T1007' 'T1008' 'T1010' 'T1011' 'T1012' 'T1014'
 'T1016' 'T1018' 'T1020' 'T1021' 'T1027' 'T1028' 'T1029' 'T1030' 'T1033'
 'T1035' 'T1036' 'T1037' 'T1039' 'T1040' 'T1041' 'T1043' 'T1046' 'T1047'
 'T1048' 'T1049' 'T1052' 'T1053' 'T1055' 'T1056' 'T1057' 'T1059' 'T1060'
 'T1061' 'T1064' 'T1068' 'T1069' 'T1070' 'T1071' 'T1072' 'T1074' 'T1078'
 'T1080' 'T1081' 'T1082' 'T1083' 'T1084' 'T1085' 'T1086' 'T1087' 'T1090'
 'T1091' 'T1092' 'T1095' 'T1098' 'T1102' 'T1104' 'T1105' 'T1106' 'T1110'
 'T1111' 'T1112' 'T1113' 'T1114' 'T1115' 'T1119' 'T1120' 'T1124' 'T1125'
 'T

In [38]:
(np.sum(one_hot_encoded, axis=0))

array([5427513,     875,      40,     598,      35,     616,     943,
             8,      50,       8,      50,       2,      27,       1,
            65,      66,      36,     113,     247,     978,      86,
             6,      91,     388,      35,    1120,     215,      47,
           943,      10,       1,      98,       2,      18,      29,
           381,   42639,   12667,     254,   12020,   40646,     207,
           755,   25540,       6,   12826,   13161,     559,   23305,
         32719,      28,       3,   40373,    2953,      55,    5741,
           186,     222,     850,   13722,    1743,   78462,   11683,
         22565,   11094,     341,    9005,   10798,    1813,    6367,
         89789,     216,       4,     509,    1153,    8063,    1259,
         97264,     379,    3603, 1470258,    3264,      36,    6081,
           657,       1,       4,     570,   79389,    2891,     654,
           254,     361,   51822,    1211,     366,   28519,   40436,
        190591,    1

In [60]:
aux_SER.info()

<class 'pandas.core.series.Series'>
Index: 9465497 entries, 0 to 9516836
Series name: MitreTechniques
Non-Null Count    Dtype 
--------------    ----- 
9465497 non-null  object
dtypes: object(1)
memory usage: 144.4+ MB


1st approach analyze high null columns


In [22]:
cols_to_fill = list(df_train.columns.difference(("OSVersion", "OSFamily")))

In [39]:
# Currently not used because all of the NULL values were dropped 
pd.set_option('future.no_silent_downcasting', True)
df_train[cols_to_fill] = df_train[cols_to_fill].fillna(value = "None")
df_test[cols_to_fill] = df_test[cols_to_fill].fillna(value = "None")

In [37]:
f_exp = df_train["IncidentGrade"].value_counts(normalize = 1, sort = 0)
f_exp#['BenignPositive','TruePositive','FalsePositive']

IncidentGrade
TruePositive      0.351034
FalsePositive     0.214671
BenignPositive    0.434295
Name: proportion, dtype: float64

In [55]:
target_unique_values = ['BenignPositive','TruePositive','FalsePositive']

In [ ]:
f_obs = df_train.groupby(['SuspicionLevel'])['IncidentGrade'].value_counts(normalize=0, dropna = 0, sort = False)
f_obs

In [ ]:
suslvls_indexes = ["Incriminated","None", "Suspicious"]
suslvls_indexes

In [ ]:
chisquare_test( f_obs, f_exp,suslvls_indexes, target_unique_values )
    

In [ ]:
for col in high_null_cols:
    print(f"{col} : {df_train[df_train[col].notna()]['IncidentGrade'].value_counts()}\n")

In [ ]:
for col in high_null_cols:
    print(f"{col} : {df_train.groupby(col)['IncidentGrade'].value_counts()}")

In [ ]:
df_train['MitreTechniques'].value_counts() 

In [66]:
check_df.info()

<class 'pandas.core.series.Series'>
RangeIndex: 9516837 entries, 0 to 9516836
Series name: MitreTechniques
Non-Null Count    Dtype 
--------------    ----- 
4048451 non-null  object
dtypes: object(1)
memory usage: 72.6+ MB


2nd aproach delete cols with high null values

In [22]:
df_train = df_train.drop(labels = high_null_cols, axis = 1)
df_test = df_test.drop(labels = high_null_cols, axis = 1)
df_train.shape, df_test.shape

((9465497, 37), (4147992, 37))

Find out the cardinality for columns 


In [19]:
for i in df_train.columns:
    counts = df_train[i].value_counts()
    cardinality = sum(counts > 1000)
    print(i, cardinality)

Id 606
OrgId 752
IncidentId 628
AlertId 16
DetectorId 437
AlertTitle 471
Category 17
IncidentGrade 3
EntityType 19
EvidenceRole 2
DeviceId 17
Sha256 58
IpAddress 146
Url 53
AccountSid 52
AccountUpn 71
AccountObjectId 49
AccountName 55
DeviceName 17
NetworkMessageId 2
RegistryKey 2
RegistryValueName 1
RegistryValueData 2
ApplicationId 11
ApplicationName 11
OAuthApplicationId 1
FileName 51
FolderPath 40
ResourceIdName 1
OSFamily 4
OSVersion 5
SuspicionLevel 1
CountryCode 53
State 85
City 84
year 1
month 2
day 29
hour 24
minute 60
second 60


There are a couple of really high cardinality data like all of columns containg ID in their name, maybe ther is some similarity beetwen them and it will be a good idea to try to group them into simmmilar clusters using kmodes

In [ ]:
#Not existing due to a high number of NULL values 
# df_train["Roles"].value_counts() # embeddings, as it is constant

In [20]:
#Not existing due to a high number of NULL values 
df_train["LastVerdict"].value_counts() # user oridinal encoding for cross entropy

LastVerdict
Suspicious                                            1402997
Malicious                                              433359
NoThreatsFound                                         397718
DomainPII_50d8b4a941c26b89482c94ab324b5a274f9ced66        128
DomainPII_9207384283ce115db5a590dd9ca5de21e5e99df2         63
Name: count, dtype: int64

In [26]:
# Not existing due to a high number of NULL values 
# df_train["SuspicionLevel"].value_counts() # heavy imbalance, remove


SuspicionLevel  IncidentGrade 
Incriminated    BenignPositive        418
                FalsePositive          31
                TruePositive           98
None            BenignPositive    3207263
                FalsePositive     1810542
                TruePositive      3010222
Suspicious      BenignPositive     903136
                FalsePositive      221394
                TruePositive       312393
Name: count, dtype: int64

As we can see besides high Null values the data result is not random, therefore it might be usefull for our model to use   

In [14]:
df_train.groupby(['EntityType'])['Category'].value_counts(normalize=0, dropna = 0)

EntityType             Category           
ActiveDirectoryDomain  Discovery                10
                       CredentialAccess          1
AmazonResource         SuspiciousActivity        4
                       InitialAccess             2
AzureResource          InitialAccess          3214
                                              ... 
User                   UnwantedSoftware       2717
                       PrivilegeEscalation    1676
                       CredentialStealing      291
                       Exploit                 267
                       WebExploit               21
Name: count, Length: 272, dtype: int64

In [23]:
df_train.groupby(['LastVerdict'])['IncidentGrade'].value_counts(normalize=True, dropna = 0)

LastVerdict                                         IncidentGrade 
DomainPII_50d8b4a941c26b89482c94ab324b5a274f9ced66  TruePositive      0.562500
                                                    BenignPositive    0.328125
                                                    FalsePositive     0.109375
DomainPII_9207384283ce115db5a590dd9ca5de21e5e99df2  TruePositive      1.000000
Malicious                                           BenignPositive    0.562303
                                                    TruePositive      0.313922
                                                    FalsePositive     0.123775
NoThreatsFound                                      BenignPositive    0.531928
                                                    TruePositive      0.292502
                                                    FalsePositive     0.175571
None                                                BenignPositive    0.384736
                                                    TruePositive

All values for 
IncidentGrade
BenignPositive    0.434295
TruePositive      0.351034
FalsePositive     0.214671




Compared to the percantages of the target value the difference here is substanitall


In [9]:
df_train.groupby(['OSFamily','OSVersion'])['IncidentGrade'].value_counts(normalize = 1)

OSFamily  OSVersion  IncidentGrade 
0         0          BenignPositive    0.639062
                     FalsePositive     0.205349
                     TruePositive      0.155589
          1          BenignPositive    0.749389
                     FalsePositive     0.150367
                                         ...   
3         66         TruePositive      0.142857
4         36         TruePositive      1.000000
5         66         BenignPositive    0.430034
                     TruePositive      0.355093
                     FalsePositive     0.214873
Name: proportion, Length: 104, dtype: float64

It doesn't look like one of the OS's are more prone to attacks due to some safety problems. There is substantiall difference in volume of attacks per OS the difference might be due to popularity (there are more windows OSs than MacOSs and there are more windows 10s than windows XPs and so on)

In [ ]:
# df_train.groupby('EmailClusterId')['IncidentGrade'].value_counts()
# df_train.columns

In [7]:
df_train["Id"].value_counts()

Id
601295424398     29997
609885356628     20512
1666447311459    11646
1417339210481    10228
635655161844     10194
                 ...  
738734376692         1
1314259993288        1
1348619731756        1
412316864851         1
463856469734         1
Name: count, Length: 730778, dtype: int64

In [11]:
df_train["IncidentId"].value_counts()

IncidentId
0         29997
2         20525
7         12252
9         11656
14        10228
          ...  
272487        1
591046        1
586536        1
231047        1
591744        1
Name: count, Length: 466151, dtype: int64

In [10]:
df_train["AlertId"].value_counts()

AlertId
0          11292
3           2681
2           2398
5           1482
6           1473
           ...  
1664650        1
1691385        1
1663669        1
1704295        1
1701413        1
Name: count, Length: 1265644, dtype: int64

In [ ]:
df_train["EntityType"].value_counts() # use embeddings

In [ ]:
#Not existing due to a high number of NULL values 
# df_train["ResourceType"].value_counts() # small amount of data, remove
df_train["ResourceIdName"].value_counts() # small amount of data, remove

In [ ]:
df_train["IncidentGrade"].value_counts()  # user oridinal encoding for cross entropy

In [ ]:
df_train["EvidenceRole"].value_counts() # one hot

In [ ]:
df_train["EntityType"].value_counts() # use embeddings

In [ ]:
df_train["Category"].value_counts()

In [ ]:
df_train["Sha256"].value_counts()  #Not sure if this is relevant from safety point of view or from attacker target selection POV #there might be removed 

In [ ]:
df_train["OSFamily"].value_counts()

In [25]:
#Not existing due to a high number of NULL values 
# sum(df_train["ThreatFamily"].value_counts() > 1000 ) # too many categories, remove

In [26]:
# if not mentioned explicitly, target encoding should be used for columns with cardinality > 3 and one hot otherwise

In [27]:
#Not existing due to a high number of NULL values 
#df_train["ActionGrouped"].value_counts()

In [20]:
object_columns = df_train.columns[df_train.dtypes == 'object'].tolist()
object_columns.remove("IncidentGrade")
object_columns

['OrgId',
 'IncidentId',
 'AlertId',
 'Category',
 'EntityType',
 'EvidenceRole',
 'SuspicionLevel']

In [15]:
# for column in object_columns: 
#     print(df_train.groupby('IncidentGrade')[column].value_counts())

In [32]:
for column in df_train.select_dtypes([object]):
    unique_values = df_train[column].unique()
    print(f"Column: {column}")
    print(f"3 Unique Values: {unique_values[:3]}")
    print(f"Number of unique values: {len(unique_values)}")
    print(sum(df_train[column].value_counts() > 1000 ))
    print(f"Number of NULL values: {df_train[column].isnull().sum()}")
    print("-" * 30)

Column: OrgId
3 Unique Values: [0 88 809]
Number of unique values: 5340
752
Number of NULL values: 0
------------------------------
Column: IncidentId
3 Unique Values: [612 326 58352]
Number of unique values: 448901
628
Number of NULL values: 0
------------------------------
Column: AlertId
3 Unique Values: [123247 210035 712507]
Number of unique values: 1248789
16
Number of NULL values: 0
------------------------------
Column: Category
3 Unique Values: ['InitialAccess' 'Exfiltration' 'CommandAndControl']
Number of unique values: 20
17
Number of NULL values: 0
------------------------------
Column: IncidentGrade
3 Unique Values: ['TruePositive' 'FalsePositive' 'BenignPositive']
Number of unique values: 3
3
Number of NULL values: 0
------------------------------
Column: EntityType
3 Unique Values: ['Ip' 'User' 'Url']
Number of unique values: 33
19
Number of NULL values: 0
------------------------------
Column: EvidenceRole
3 Unique Values: ['Related' 'Impacted']
Number of unique values:

In [33]:
# for column in object_columns: 
# print(df_train[df_train['IncidentGrade'].isna()][object_columns])

In [17]:
list_of_columns = list(df_train.columns)
list_of_columns

['Id',
 'OrgId',
 'IncidentId',
 'AlertId',
 'DetectorId',
 'AlertTitle',
 'Category',
 'IncidentGrade',
 'EntityType',
 'EvidenceRole',
 'DeviceId',
 'Sha256',
 'IpAddress',
 'Url',
 'AccountSid',
 'AccountUpn',
 'AccountObjectId',
 'AccountName',
 'DeviceName',
 'NetworkMessageId',
 'RegistryKey',
 'RegistryValueName',
 'RegistryValueData',
 'ApplicationId',
 'ApplicationName',
 'OAuthApplicationId',
 'FileName',
 'FolderPath',
 'ResourceIdName',
 'OSFamily',
 'OSVersion',
 'CountryCode',
 'State',
 'City',
 'year',
 'month',
 'day',
 'hour',
 'minute',
 'second']

Change ORGID and incidentID to Frequency encoder and alertID to target encoder

In [23]:
X_train = df_train.drop(columns = ["IncidentGrade"])
X_test = df_test.drop(columns = ["IncidentGrade"])


y_train = df_train["IncidentGrade"]
y_test = df_test["IncidentGrade"]
X_train.shape


(9465497, 36)

In [58]:
X_train["AlertId"]

0          123247
1          210035
2          712507
3          774301
4          188041
            ...  
9516832    496631
9516833    748366
9516834      7425
9516835      8973
9516836    644106
Name: AlertId, Length: 9465497, dtype: object

In [24]:
# y_train = OrdinalEncoder().transform(y_train)
# y_test = OrdinalEncoder().transform(y_test)
# y_train
# Oridnal encoding of target value
priority_map = {
    "FalsePositive": 0,
    "BenignPositive": 1,
    "TruePositive": 2
}
y_train = y_train.map(priority_map)
y_test = y_test.map(priority_map)
y_train

0          2
1          0
2          0
3          1
4          2
          ..
9516832    2
9516833    1
9516834    1
9516835    1
9516836    1
Name: IncidentGrade, Length: 9465497, dtype: int64

In [60]:
# for col in df_train.columns[:4]:
#     print(df_train[col].value_counts())


In [61]:
y_train.shape

(9465497,)

In [25]:
# X_encode = X_train.sample(frac=0.25)
# y_encode = y_train[X_encode.index]
# X_train = X_train.drop(X_encode.index)
# y_train = y_train[X_train.index]
# y_train.shape, y_encode.shape
X_train, X_encode, y_train, y_encode = train_test_split(X_train, y_train, test_size=0.32)
print("Training shapes:", X_train.shape, y_train.shape)

Training shapes: (6436537, 36) (6436537,)


In [26]:
high_card_cols = ["Category", "EntityType"]#,  "Roles", "AlertId"]#"ActionGranular",
# encoder_train = TargetEncoder(cols = high_card_cols, smoothing=10, min_samples_leaf = 20).fit(X_train, y_train) 
# # Fit and transform
# X_train = encoder_train.transform(X_train)
# X_train[high_card_cols]
enc = TargetEncoder(cols = high_card_cols, smoothing=11, min_samples_leaf = 22)
enc.fit(X_encode, y_encode)
X_train = enc.transform(X_train)
X_test = enc.transform(X_test)
X_train.shape

(6436537, 36)

In [64]:
X_train[high_card_cols]

,Category,EntityType
3155928,0.826802,1.224981
6652547,1.307054,1.247862
5991337,1.307054,1.197109
2491076,0.907593,0.780123
6953231,1.307054,1.197109
...,...,...
4501987,1.307054,0.922762
5413993,1.307054,1.101192
8549041,0.826802,0.922762
280513,0.837156,0.780123


In [27]:
# Columns to encode
columns_to_freq_encode = ["OrgId", "IncidentId", "AlertId"]

# Apply frequency encoding
for col in columns_to_freq_encode:
    freq_train = X_train[col].value_counts(normalize=True)  # Get frequencies (normalized)
    freq_test = X_test[col].value_counts(normalize=True)
    X_train[col] = X_train[col].map(freq_train)  # Map frequencies
    X_test[col] = X_test[col].map(freq_test)

In [25]:
X_train.shape

(6436537, 40)

In [26]:
#Ommited for now due to dropped HIgh NULL number columns 
# columns_to_OH_encode = ["LastVerdict","EvidenceRole","ActionGrouped"]
# OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output = False)

# OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[columns_to_OH_encode])) 
# OH_cols_test = pd.DataFrame(OH_encoder.transform(X_test[columns_to_OH_encode]))
# OH_cols_train 

In [27]:
X_train.shape

(6436537, 39)

In [47]:
# print(X_train.index.equals(OH_cols_train.index)) 
# X_train.columns

In [48]:
# X_train = pd.concat([X_train.reset_index(drop=True), OH_cols_train.reset_index(drop=True)], axis=1).drop(columns = columns_to_OH_encode )
# # X_train = pd.concat([X_train, OH_cols_train], axis=1, ignore_index = True).drop(columns = columns_to_OH_encode )
# # X_test = pd.concat([X_test, OH_cols_test], axis=1, ignore_index = True).drop(columns = columns_to_OH_encode )
# X_test = pd.concat([X_test.reset_index(drop=True), OH_cols_test.reset_index(drop=True)], axis=1).drop(columns = columns_to_OH_encode )

In [25]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6436537 entries, 1065311 to 6026867
Data columns (total 39 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Id                  int64  
 1   OrgId               float64
 2   IncidentId          float64
 3   AlertId             float64
 4   DetectorId          int64  
 5   AlertTitle          int64  
 6   Category            float64
 7   EntityType          float64
 8   EvidenceRole        object 
 9   DeviceId            int64  
 10  Sha256              int64  
 11  IpAddress           int64  
 12  Url                 int64  
 13  AccountSid          int64  
 14  AccountUpn          int64  
 15  AccountObjectId     int64  
 16  AccountName         int64  
 17  DeviceName          int64  
 18  NetworkMessageId    int64  
 19  RegistryKey         int64  
 20  RegistryValueName   int64  
 21  RegistryValueData   int64  
 22  ApplicationId       int64  
 23  ApplicationName     int64  
 24  OAuthApplicationId  int

In [28]:
cols_to_delete = ["AccountName", "DeviceName", "year",]   #"AntispamDirection","EmailClusterId, "EvidenceRole",
X_train = X_train.drop(columns = cols_to_delete) 
X_test = X_test.drop(columns = cols_to_delete) 

In [ ]:
X_train.info()

In [ ]:
X_train.shape, y_train.shape, X_train.info()

In [29]:
int_cols = list(X_train.select_dtypes(include=['int64', 'object', 'float64']).columns)
int_cols

['Id',
 'OrgId',
 'IncidentId',
 'AlertId',
 'DetectorId',
 'AlertTitle',
 'Category',
 'EntityType',
 'EvidenceRole',
 'DeviceId',
 'Sha256',
 'IpAddress',
 'Url',
 'AccountSid',
 'AccountUpn',
 'AccountObjectId',
 'NetworkMessageId',
 'RegistryKey',
 'ApplicationId',
 'ApplicationName',
 'FileName',
 'FolderPath',
 'OSFamily',
 'OSVersion',
 'SuspicionLevel',
 'CountryCode',
 'State',
 'City']

In [30]:
int_cols.remove("Category")

In [31]:
int_cols.remove("EntityType")

In [32]:
int_cols.remove("OrgId")
int_cols.remove("IncidentId")
int_cols.remove("AlertId")



In [33]:
int_cols

['Id',
 'DetectorId',
 'AlertTitle',
 'EvidenceRole',
 'DeviceId',
 'Sha256',
 'IpAddress',
 'Url',
 'AccountSid',
 'AccountUpn',
 'AccountObjectId',
 'NetworkMessageId',
 'RegistryKey',
 'ApplicationId',
 'ApplicationName',
 'FileName',
 'FolderPath',
 'OSFamily',
 'OSVersion',
 'SuspicionLevel',
 'CountryCode',
 'State',
 'City']

In [34]:
for col_name in int_cols:
    X_train[col_name] = X_train[col_name].astype("category")
    X_test[col_name] = X_test[col_name].astype("category")

In [35]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6436537 entries, 8055258 to 90793
Data columns (total 33 columns):
 #   Column            Dtype   
---  ------            -----   
 0   Id                category
 1   OrgId             float64 
 2   IncidentId        float64 
 3   AlertId           float64 
 4   DetectorId        category
 5   AlertTitle        category
 6   Category          float64 
 7   EntityType        float64 
 8   EvidenceRole      category
 9   DeviceId          category
 10  Sha256            category
 11  IpAddress         category
 12  Url               category
 13  AccountSid        category
 14  AccountUpn        category
 15  AccountObjectId   category
 16  NetworkMessageId  category
 17  RegistryKey       category
 18  ApplicationId     category
 19  ApplicationName   category
 20  FileName          category
 21  FolderPath        category
 22  OSFamily          category
 23  OSVersion         category
 24  SuspicionLevel    category
 25  CountryCode       c

In [ ]:
params = {
    'objective': 'multi:softmax',  # for ordinal classification
    'num_class': 4,  # replace N with your number of ordered categories
    'tree_method': 'hist',  # essential for large datasets
    'max_cat_to_onehot': 4,  # One-hot encode if cardinality ≤ 4, else use optimal partitioning
    'grow_policy': 'lossguide',  # better for large data
    'max_depth': 7,  # conservative depth for ordinal data
    'learning_rate': 0.05,  # smaller for better ordinal separation
    'subsample': 0.8,  # random row selection
    'colsample_bytree': 0.7,  # feature randomization
    'min_child_weight': 80,  # higher for large dataset
    'reg_lambda': 1.5,  # L2 regularization
    'reg_alpha': 0.5,  # L1 regularization
    'max_bin': 200,  # balance speed and accuracy
    #'n_estimators': 2500,  # will use early stopping
    'eval_metric': 'mlogloss',  # for multiclass
    #'enable_categorical': True,  # set True if using categorical features
    'tree_method': 'hist',
    'device' : 'cuda'
}

dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True )
dtest = xgb.DMatrix(X_test, label=y_test, enable_categorical=True)

# dtrain = X_train
# dtest = X_test 

model = xgb.train(
    params,
    dtrain,
    num_boost_round=2500,
    evals=[(dtrain, 'train'), (dtest, 'val')],
    early_stopping_rounds=50,
    verbose_eval = True
)

# callbacks = [
#     xgb.callback.EarlyStopping(rounds=50),
#     xgb.callback.EvaluationMonitor(period=100),
# #     xgb.callback.LearningRateScheduler(custom_decay_fn)
# ]

[0]	train-mlogloss:1.32594	val-mlogloss:1.33979
[1]	train-mlogloss:1.26993	val-mlogloss:1.29650
[2]	train-mlogloss:1.22088	val-mlogloss:1.26425
[3]	train-mlogloss:1.16917	val-mlogloss:1.22004
[4]	train-mlogloss:1.12593	val-mlogloss:1.18873
[5]	train-mlogloss:1.08448	val-mlogloss:1.15843
[6]	train-mlogloss:1.04442	val-mlogloss:1.12893
[7]	train-mlogloss:1.00769	val-mlogloss:1.09628
[8]	train-mlogloss:0.97644	val-mlogloss:1.06935
[9]	train-mlogloss:0.94513	val-mlogloss:1.04537
[10]	train-mlogloss:0.91662	val-mlogloss:1.02805
[11]	train-mlogloss:0.89259	val-mlogloss:1.00735
[12]	train-mlogloss:0.86603	val-mlogloss:0.98683
[13]	train-mlogloss:0.84252	val-mlogloss:0.96686
[14]	train-mlogloss:0.81892	val-mlogloss:0.94702
[15]	train-mlogloss:0.79729	val-mlogloss:0.92996
[16]	train-mlogloss:0.77391	val-mlogloss:0.91159
[17]	train-mlogloss:0.75417	val-mlogloss:0.89530
[18]	train-mlogloss:0.73375	val-mlogloss:0.87738
[19]	train-mlogloss:0.71303	val-mlogloss:0.86080
[20]	train-mlogloss:0.69491	va

params = {
    'objective': 'multi:softmax',  # for ordinal classification
    'num_class': 4,  # replace N with your number of ordered categories
    'tree_method': 'hist',  # essential for large datasets
    'max_cat_to_onehot': 4,  # One-hot encode if cardinality ≤ 4, else use optimal partitioning
    'grow_policy': 'lossguide',  # better for large data
    'max_depth': 8,  # conservative depth for ordinal data
    'learning_rate': 0.05,  # smaller for better ordinal separation
    'subsample': 0.8,  # random row selection
    'colsample_bytree': 0.7,  # feature randomization
    'min_child_weight': 70,  # higher for large dataset
    'reg_lambda': 1.5,  # L2 regularization
    'reg_alpha': 0.5,  # L1 regularization
    'max_bin': 256,  # balance speed and accuracy
    #'n_estimators': 2500,  # will use early stopping
    'eval_metric': 'mlogloss',  # for multiclass
    #'enable_categorical': True,  # set True if using categorical features
    'tree_method': 'hist',
    'device' : 'cuda'
}

[0]	train-mlogloss:1.30925	val-mlogloss:1.34542
[1]	train-mlogloss:1.23818	val-mlogloss:1.31238
[2]	train-mlogloss:1.17284	val-mlogloss:1.27785
[3]	train-mlogloss:1.11697	val-mlogloss:1.25042
[4]	train-mlogloss:1.06181	val-mlogloss:1.22281
[5]	train-mlogloss:1.02261	val-mlogloss:1.20177

In [ ]:
X_train.info()


In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Get predictions (replace with your actual model and data)
y_pred = model.predict(dtrain)  # For DMatrix
# y_pred = model.predict(X_test)  # For DataFrame input

# Create confusion matrix
cm = confusion_matrix(y_train, y_pred)
num = y_train.shape[0]
sraka = cm/num
# Plot with Seaborn
sns.set(font="Palatino")
plt.figure(figsize=(10, 7))
sns.heatmap(sraka, annot=True, fmt='.2f', cmap='Blues',
           xticklabels=priority_map,
            yticklabels=priority_map            
)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix - Incident Grade Prediction')
plt.savefig("chujdupaicpa.png")
plt.show()

NameError: name 'model' is not defined